In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
#p10_GG.csv is created in ArcGIS through spatial join of p10 polygons and Draft Blueprint growth geography polygons.

#p10                  : M:\Data\GIS layers\UrbanSim smelt\2020 03 12\smelt.gdb
#Growth Geography (GG): M:\Data\GIS layers\Blueprint Land Use Strategies\Draft Blueprint_v4_2-11-2020_Spatial Data
#Spatial join rule    : Centers of p10 polygons fall into "Growth Geography (GG)" polygons
#Spatial join output  : M:\Data\GIS layers\Blueprint Land Use Strategies\Blueprint Land Use Strategies.gdb\p10_GG

#Then run script 'https://github.com/BayAreaMetro/petrale/blob/master/basemap/import_filegdb_layers.py' 
#to export the 'p10_GG' layer to .csv format.


if os.getenv('USERNAME')    =='ywang':
    WORK_DIR                = 'M:\\Data\\GIS layers\\Blueprint Land Use Strategies'

In [3]:
p10_gg_file = os.path.join(WORK_DIR, 'p10_GG_v4_2-11-2020.csv')
p10_gg_raw = pd.read_csv(p10_gg_file)
print('Read {:,} rows from p10_gg_file'.format(len(p10_gg_raw)))

C:\Users\ywang\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (25) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Read 1,956,208 rows from p10_gg_file


In [4]:
# Keep needed fields and name them as needed
p10_gg = p10_gg_raw[['PARCEL_ID','geom_id_s','manual_county','County','ACRES','PDA_Name','Jurisdicti','PDA_Change','Join_Key','Designatio']]
p10_gg.rename(columns = {'Jurisdicti': 'Jurisdiction',
                         'manual_county': 'County_ID',
                         'Designatio': 'Designation'}, inplace = True)

C:\Users\ywang\Anaconda3\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [5]:
# Get unique PDAs
pdas = p10_gg[['PDA_Name','County_ID','Jurisdiction']].drop_duplicates()
pdas = pdas.loc[(pdas.PDA_Name.notnull()) & (pdas.PDA_Name != ' ')]
pdas.sort_values(by = ['County_ID','Jurisdiction','PDA_Name'], inplace = True)
print('There are {:,} unique PDAs in {} jurisdictions'.format(len(pdas), len(pdas.Jurisdiction.unique())))
display(pdas.head())

There are 221 unique PDAs in 76 jurisdictions


<bound method NDFrame.head of                                           PDA_Name  County_ID  \
6106                             Naval Air Station          1   
3239                           Northern Waterfront          1   
28831    San Pablo & Solano Mixed Use Neighborhood          1   
3479                                Adeline Street          1   
6057                                      Downtown          1   
...                                            ...        ...   
2020                                     Core Area         97   
5511              Airport Industrial Specific Plan         97   
9149   Santa Rosa Avenue Priority Development Area         97   
5507     Sonoma County: Sonoma Valley, The Springs         97   
4712      Station Area/Downtown Specific Plan Area         97   

                Jurisdiction  
6106                 Alameda  
3239                 Alameda  
28831                 Albany  
3479                Berkeley  
6057                Berkeley  
...

In [6]:
# Assign index to PDAs by jurisdiction
pdas['idx'] = pdas.groupby(['County_ID','Jurisdiction']).cumcount()+1
display(pdas.head())

,PDA_Name,County_ID,Jurisdiction,idx
6106,Naval Air Station,1,Alameda,1
3239,Northern Waterfront,1,Alameda,2
28831,San Pablo & Solano Mixed Use Neighborhood,1,Albany,1
3479,Adeline Street,1,Berkeley,1
6057,Downtown,1,Berkeley,2


In [7]:
# Join the index back to p10 parcels
p10_gg_idxed = p10_gg.merge(pdas, 
                          on = ['PDA_Name', 'County_ID', 'Jurisdiction'],
                          how = 'left')
display(p10_gg_idxed.head())
p10_gg_idxed.to_csv(os.path.join(WORK_DIR, 'p10_gg_idxed.csv'),index = False)     

,PARCEL_ID,geom_id_s,County_ID,County,ACRES,PDA_Name,Jurisdiction,PDA_Change,Join_Key,Designation,idx
0,229116.0,10305106092872,1,NaN,3.360520,NaN,NaN,NaN,NaN,NaN,NaN
1,244166.0,11107351665227,1,NaN,1.294423,NaN,NaN,NaN,NaN,NaN,NaN
2,202378.0,11030175960628,1,NaN,14.993605,NaN,NaN,NaN,NaN,NaN,NaN
3,2004420.0,6381677629073,97,NaN,316.247146,NaN,NaN,NaN,NaN,NaN,NaN
4,340332.0,314875459798,1,NaN,0.621275,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# Simple stats
print('Number of PDAs by Jurisdiction:')
pda_count_juris = pdas.groupby(['County_ID','Jurisdiction'])['PDA_Name'].count().reset_index()
pda_count_juris.rename(columns = {'PDA_Name':'Number of PDAs'}, inplace = True)
display(pda_count_juris)
pda_count_juris.to_csv(os.path.join(WORK_DIR,'pda_count_juris.csv'),index = False)

Number of PDAs by Jurisdiction:


,County_ID,Jurisdiction,Number of PDAs
0,1,Alameda,2
1,1,Albany,1
2,1,Berkeley,7
3,1,Dublin,3
4,1,Emeryville,1
...,...,...,...
75,97,Rohnert Park,2
76,97,Santa Rosa,5
77,97,Sebastopol,1
78,97,Unincorporated Sonoma,3


In [10]:
print('Parcels Acreage by PDAs:')
p10_acr_pda = p10_gg_idxed.groupby(['County','Jurisdiction','PDA_Name'])['ACRES'].sum().reset_index()
display(p10_acr_pda)
p10_acr_pda.to_csv(os.path.join(WORK_DIR,'p10_acr_pda.csv'),index = False)

Parcels Acreage by PDAs:


,County,Jurisdiction,PDA_Name,ACRES
0,,,,32540.620126
1,Alameda,Alameda,Naval Air Station,593.477511
2,Alameda,Alameda,Northern Waterfront,288.534500
3,Alameda,Albany,San Pablo & Solano Mixed Use Neighborhood,44.234891
4,Alameda,Berkeley,Adeline Street,34.249892
...,...,...,...,...
211,Sonoma,Sebastopol,Core Area,609.460444
212,Sonoma,Unincorporated Sonoma,Airport Industrial Specific Plan,721.203403
213,Sonoma,Unincorporated Sonoma,Santa Rosa Avenue Priority Development Area,423.699519
214,Sonoma,Unincorporated Sonoma,"Sonoma County: Sonoma Valley, The Springs",170.834713
